In [67]:
from auxFuncs import *

In [68]:
print("...ok! Python3 loaded.")

...ok! Python3 loaded.


In [69]:
now = datetime.now().strftime("%m_%d_%Y_%H_%M_%S")
sourceDir = "/Users/jordan/ThesisMARL/SURPL-V2/JJ_Juypter/results"
fileOut = open(sourceDir + "/PPO.txt", "w+")
tfLogs = sourceDir + "/logs"
now

'02_08_2022_21_27_23'

In [70]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [71]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(sourceDir + "/logs", histogram_freq=1)

In [72]:
# ------------------------------
#       SMART BUILDING
# ------------------------------
class SmartBuildingEnv(Env):
    def __init__(self):
        self.viewer = None
        self.load = 0
        self.demand = []
        self.timeStep = 0
        self.deltaUtilization = 0
        self.penalty = 0
        self.action_space = None
        self.observation_space = None
        self.reset()
        
    def reset(self):
        # self.load = 0
        # MOVE TO 24 HOURS FOR INITAL RESULTS NOW THAT PRELIM RESULTS ESTABLISHED
        self.demand = [3, 1, 1]
        # self.demand = np.array([3,1,1])
        # self.demand = [3, 2, 1]
        # self.timeStep +=1 == self.demand[self.timeStep]
        self.timeStep = 0
        self.deltaUtilization = abs(self.demand[self.timeStep] - self.load)
        self.penalty = (self.deltaUtilization) ** 2
        self.action_space = Box(low=0, high=self.demand[self.timeStep], shape=(1,), dtype=float)
        # OBSERVATION SPACE CAN ALSO INCLUDE DEMAND CHARGE, TIME, ETC -> 1 INSUFFICIENT FOR MORE RESULTS
        self.observation_space = Box(low=0, high=self.deltaUtilization, shape=(1,), dtype=float)
        # return np.array(0)
        return np.array([0])
        # pass
        # return state
    
    def step(self, action):
        info = {}
        reward = 0
        done:bool = False
        self.load = action[0]

        print("SB stepping: ", round(action[0], 2))
        print("*"*25)
        print("*"*25)
        print("SB stepping: ", round(action[0], 2), file=fileOut)
        print("*"*25, file=fileOut)
        print("*"*25, file=fileOut)
        

        print("Timestep: ", self.timeStep)
        print("-"*5)
        print("Load: ", round(self.load, 2))
        print("Demand: ", self.demand[self.timeStep])
        print("Timestep: ", self.timeStep, file=fileOut)
        print("-"*5, file=fileOut)
        print("Load: ", round(self.load, 2), file=fileOut)
        print("Demand: ", self.demand[self.timeStep], file=fileOut)


        self.deltaUtilization = abs(self.demand[self.timeStep] - self.load)
        self.penalty = (self.deltaUtilization) ** 2
        
        observation = self.demand[self.timeStep]
        
        print("Penalty: ", round(self.penalty, 2))
        print("Delta ", round(self.deltaUtilization, 2))
        print("Observation: ", observation)
        print("-"*25)
        print("Penalty: ", round(self.penalty, 2), file=fileOut)
        print("Delta ", round(self.deltaUtilization, 2), file=fileOut)
        print("Observation: ", observation, file=fileOut)
        print("-"*25, file=fileOut)
        
        # STILL NEED PEAK DEMAND IN REWARD
        # HOW DO WE CONTRIBUTE TO THIS REWARD?
        reward = 0
        if self.load != 0:
            reward -= self.load + self.penalty
        else:
            reward -= self.load + 1 + self.penalty
            
        # REWARD CAN BE DIFFERENCE BETWEEN COSTS

        self.timeStep += 1
        done = True if self.timeStep > 2 else done
            
        
        return np.array([observation]), reward, done, info
    
    def render(self, mode="human"):
        screen_h = 600
        screen_w = 1000
        if self.viewer is None:
            self.viewer = rendering.Viewer(screen_w, screen_h, "SB")
            
        return self.viewer.render(return_rgb_array=mode == "rgb_array")
    
    def close(self):
        if self.viewer:
            self.viewer.close()
            self.viewer = None

In [73]:
env = SmartBuildingEnv()
# env = DummyVecEnv([lambda: env])
batch_size = 500
ep_per_batch = 10 * batch_size

In [74]:
check_env(env)

SB stepping:  2.96
*************************
*************************
Timestep:  0
-----
Load:  2.96
Demand:  3
Penalty:  0.0
Delta  0.04
Observation:  3
-------------------------


AssertionError: The observation returned by `step()` method must be a numpy array

In [ ]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=tfLogs)
model.learn(total_timesteps=ep_per_batch, reset_num_timesteps=False)

In [ ]:
obs = env.reset() 

for i in range(batch_size):
    action, _states = model.predict(obs, deterministic=True)
    # action = env.action_space.sample()
    
    obs, reward, done, info = env.step(action)
    # if ep_per_batch % 2000 == 0:
    # env.render()
    if done:
        obs = env.reset()

In [ ]:
# mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100, render=True)
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=1000)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

In [ ]:
model.save("results/PPO_" + "/BuildingTemp")

In [ ]:
%tensorboard --logdir {sourceDir + "/logs"}

In [ ]:
fileOut.close()